In [ ]:
# 데이터 다운로드 
import os
import urllib.parse
import requests
import re

urls=[ 
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/refs/heads/main/ch04/data/paul_graham_essay.txt", # 폴 그레이엄 에세이
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch04/data/%EB%AF%B8%EA%B5%AD%EC%9D%98%20%ED%86%B5%ED%99%94%EC%A0%95%EC%B1%85%20%EB%B3%80%ED%99%94%EA%B0%80%20%EC%99%B8%EA%B5%AD%EC%9E%90%EB%B3%B8%20%EC%9C%A0%EC%B6%9C%EC%97%90%20%EB%AF%B8%EC%B9%98%EB%8A%94%20%EC%98%81%ED%96%A5_20180528.hwp", # 미국 통화 정책 변화 파일
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/refs/heads/main/ch04/data/%EB%B2%94%EC%A3%84%20%EB%B0%9C%EC%83%9D%20%EC%9E%A5%EC%86%8C%EB%B3%84%20%ED%86%B5%EA%B3%84_2023.csv", # ‘범죄 통계 
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch04/data/%EC%9D%B8%EB%AC%B8%ED%95%99%EC%9D%98%20%EC%83%88%EB%A1%9C%EC%9A%B4%20%EC%97%AD%ED%95%A0%EC%97%90%20%EB%8C%80%ED%95%9C%20%EA%B3%A0%EC%B0%B0.pdf" # 인문학 파일
]

# 각 파일 다운로드
for url in urls:
    encoded_filename = url.split("/")[-1]  # URL에서 파일명 추출
    decoded_filename = urllib.parse.unquote(encoded_filename) # 한글 파일명 복원
    response = requests.get(url)
    if response.status_code == 200:
# 임시 파일명으로 저장
        temp_filename = "temp_download_file" + os.path.splitext(decoded_filename)[1] 
        with open(temp_filename, 'wb') as f:
            f.write(response.content)            
        os.rename(temp_filename, decoded_filename) # 파일명 변경
        print(f"완료: {decoded_filename} 다운로드 완료")
    else:
        print(f"오류: {url} 다운로드 실패 (상태 코드: {response.status_code})\n")


## PDF 파일 다루기 

In [6]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["data/인문학의 새로운 역할에 대한 고찰.pdf"])
documents = reader.load_data()


In [2]:
print(len(documents))

137


In [3]:
documents[3]

Document(id_='81167921-827a-4bda-9972-739cf91400b3', embedding=None, metadata={'page_label': '4', 'file_name': '인문학의 새로운 역할에 대한 고찰.pdf', 'file_path': 'data\\인문학의 새로운 역할에 대한 고찰.pdf', 'file_type': 'application/pdf', 'file_size': 4632243, 'creation_date': '2025-03-08', 'last_modified_date': '2024-09-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='연구요약\n인간 사회의 전 분야를 쓰나미처럼 휩쓸고 들어오는 디지털화는 개인의 생활 방식, 개인\n들끼리의 교류 방식, 더 나아가 사회가 작동하는 방식을 총체적이고 근본적으로 변화시키\n고 있는 상황이다 . 우리는 현재 디지털을 사용하는 선을 넘어서서 디지털 가운데서 살아가\n고 있다. 디지털이라는 매체가 인간 삶의 전반적 틀을 크게 좌우하는 이러한 상황은 여러 \n가지 전례 없는 현상들을 낳고 있으며 , 이러한 변화가 긍정적일지 부정적일지는 현재의 우\n리가 어떻게 대처하느냐에 달려있을 수밖에 없다. 평자에 따라서는 농업혁명 , 산업혁명 이\n상의 역사적인 전환점이라고도 평가되고 있는 현재 시점은 곧 새로운 사회구조의 원리와 \n더불어 새로운 방식의 학문이 출현하는 시점인 듯하다 .\n이 글에서는 먼저 개인정보의 문제와 젠더의

### 텍스트 분할하기 

#### 1. 토큰 단위 분할 

In [4]:
from llama_index.core.node_parser import TokenTextSplitter

In [5]:
# TokenTextSplitter 설정하기 
splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=20)

# TokenTextSplitter 적용 후 노드에 담기 
nodes = splitter.get_nodes_from_documents(documents)


In [6]:
print(len(nodes))

186


In [7]:
print(nodes[3].get_content())  

연구요약
인간 사회의 전 분야를 쓰나미처럼 휩쓸고 들어오는 디지털화는 개인의 생활 방식, 개인들끼리의 교류 방식, 더 나아가 사회가 작동하는 방식을 총체적이고 근본적으로 변화시키고 있는 상황이다. 우리는 현재 디지털을 사용하는 선을 넘어서서 디지털 가운데서 살아가고 있다. 디지털이라는 매체가 인간 삶의 전반적 틀을 크게 좌우하는 이러한 상황은 여러 가지 전례 없는 현상들을 낳고 있으며, 이러한 변화가 긍정적일지 부정적일지는 현재의 우리가 어떻게 대처하느냐에 달려있을 수밖에 없다. 평자에 따라서는 농업혁명, 산업혁명 이상의 역사적인 전환점이라고도 평가되고 있는 현재 시점은 곧 새로운 사회구조의 원리와 더불어 새로운 방식의 학문이 출현하는 시점인 듯하다.이 글에서는 먼저 개인정보의 문제와 젠더의 문제에 집중하여 디지털-빅데이터 시대가 낳은 사회적 변화의 양상을 조망하고자 했다. 인간의 생활 전반이 디지털에서 흔적으로 남고 있는 현실은 그 흔적, 즉 “trace data”를 그 자체로 “상품”으로 간주하려는 경향과 결부되고 있다. 디지털 사회로의 진입이 이미 이루어졌고 후퇴가 불가능할 뿐만 아니라 디지털-빅데이터 사회가 갖는 장점이 매우 큰 것을 고려할 때 “데이터 공동체”, 혹은 “연대적인 데이터 저장” 등 현재 사회학에서 제시되고 있는 방법들에 대해 성찰함으로써 개인데이터가 “상품”으로 직진하지 않도록 하기 위한 구체적인 방법을 찾아내기 위해 노력할 필요가 절실해 보인다.젠더의 경우 디지털-빅데이터 시대가 사회 진보의 측면에서 가능성과 한계를 동시에 안고 있음을 선명하게 보여주고 있다. 한편으로 디지털 전환 자체가 젠더에 따라 다르게 경험되고 있고, 디지털 세계에서도 기존의 남성중심적인 구조가 재생산되는 경향이 분명한 것이 현실이다. 그러나 다른 한편으로 빅데이터는 2000년대 이후 정체상태에 머물러 있는 양성평등에 한걸음 더 가까이 갈 수 있는 가능성을 내포하고 있기도 하다. 젠더 격차의 문제를 빅데이터를 통해서 입체적으로 보여줄 수가 있고, 그 결과로 구체적

#### 2.의미 단위 분할

In [8]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding


In [9]:
# embed_model 인스턴스 만들기 
embed_model = OpenAIEmbedding()

# semantic_splitter 설정하기
semantic_splitter = SemanticSplitterNodeParser(
    buffer_size=10, 
    breakpoint_percentile_threshold=95, 
    embed_model=embed_model
)
# semantic_splitter 적용 후 노드에 담기
nodes_semantic=semantic_splitter.get_nodes_from_documents(documents)


In [10]:
len(nodes_semantic)

189

In [11]:
print(nodes_semantic[3].get_content()) 

연구요약
인간 사회의 전 분야를 쓰나미처럼 휩쓸고 들어오는 디지털화는 개인의 생활 방식, 개인들끼리의 교류 방식, 더 나아가 사회가 작동하는 방식을 총체적이고 근본적으로 변화시키고 있는 상황이다. 우리는 현재 디지털을 사용하는 선을 넘어서서 디지털 가운데서 살아가고 있다. 디지털이라는 매체가 인간 삶의 전반적 틀을 크게 좌우하는 이러한 상황은 여러 가지 전례 없는 현상들을 낳고 있으며, 이러한 변화가 긍정적일지 부정적일지는 현재의 우리가 어떻게 대처하느냐에 달려있을 수밖에 없다. 평자에 따라서는 농업혁명, 산업혁명 이상의 역사적인 전환점이라고도 평가되고 있는 현재 시점은 곧 새로운 사회구조의 원리와 더불어 새로운 방식의 학문이 출현하는 시점인 듯하다.이 글에서는 먼저 개인정보의 문제와 젠더의 문제에 집중하여 디지털-빅데이터 시대가 낳은 사회적 변화의 양상을 조망하고자 했다. 인간의 생활 전반이 디지털에서 흔적으로 남고 있는 현실은 그 흔적, 즉 “trace data”를 그 자체로 “상품”으로 간주하려는 경향과 결부되고 있다. 디지털 사회로의 진입이 이미 이루어졌고 후퇴가 불가능할 뿐만 아니라 디지털-빅데이터 사회가 갖는 장점이 매우 큰 것을 고려할 때 “데이터 공동체”, 혹은 “연대적인 데이터 저장” 등 현재 사회학에서 제시되고 있는 방법들에 대해 성찰함으로써 개인데이터가 “상품”으로 직진하지 않도록 하기 위한 구체적인 방법을 찾아내기 위해 노력할 필요가 절실해 보인다.젠더의 경우 디지털-빅데이터 시대가 사회 진보의 측면에서 가능성과 한계를 동시에 안고 있음을 선명하게 보여주고 있다. 한편으로 디지털 전환 자체가 젠더에 따라 다르게 경험되고 있고, 디지털 세계에서도 기존의 남성중심적인 구조가 재생산되는 경향이 분명한 것이 현실이다. 그러나 다른 한편으로 빅데이터는 2000년대 이후 정체상태에 머물러 있는 양성평등에 한걸음 더 가까이 갈 수 있는 가능성을 내포하고 있기도 하다. 젠더 격차의 문제를 빅데이터를 통해서 입체적으로 보여줄 수가 있고, 그 결과로 구체적

#### 3.문장 단위 분할 

In [13]:
from llama_index.core.node_parser import SentenceSplitter

# semantic_splitter 설정하기
splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)

In [14]:
# semantic_splitter 적용 후 노드에 담기 
nodes_sentence = splitter.get_nodes_from_documents(documents)


In [15]:
print(len(nodes_sentence)) 

184


In [16]:
print(nodes_sentence[3].get_content()) 

연구요약
인간 사회의 전 분야를 쓰나미처럼 휩쓸고 들어오는 디지털화는 개인의 생활 방식, 개인들끼리의 교류 방식, 더 나아가 사회가 작동하는 방식을 총체적이고 근본적으로 변화시키고 있는 상황이다. 우리는 현재 디지털을 사용하는 선을 넘어서서 디지털 가운데서 살아가고 있다. 디지털이라는 매체가 인간 삶의 전반적 틀을 크게 좌우하는 이러한 상황은 여러 가지 전례 없는 현상들을 낳고 있으며, 이러한 변화가 긍정적일지 부정적일지는 현재의 우리가 어떻게 대처하느냐에 달려있을 수밖에 없다. 평자에 따라서는 농업혁명, 산업혁명 이상의 역사적인 전환점이라고도 평가되고 있는 현재 시점은 곧 새로운 사회구조의 원리와 더불어 새로운 방식의 학문이 출현하는 시점인 듯하다.이 글에서는 먼저 개인정보의 문제와 젠더의 문제에 집중하여 디지털-빅데이터 시대가 낳은 사회적 변화의 양상을 조망하고자 했다. 인간의 생활 전반이 디지털에서 흔적으로 남고 있는 현실은 그 흔적, 즉 “trace data”를 그 자체로 “상품”으로 간주하려는 경향과 결부되고 있다. 디지털 사회로의 진입이 이미 이루어졌고 후퇴가 불가능할 뿐만 아니라 디지털-빅데이터 사회가 갖는 장점이 매우 큰 것을 고려할 때 “데이터 공동체”, 혹은 “연대적인 데이터 저장” 등 현재 사회학에서 제시되고 있는 방법들에 대해 성찰함으로써 개인데이터가 “상품”으로 직진하지 않도록 하기 위한 구체적인 방법을 찾아내기 위해 노력할 필요가 절실해 보인다.젠더의 경우 디지털-빅데이터 시대가 사회 진보의 측면에서 가능성과 한계를 동시에 안고 있음을 선명하게 보여주고 있다. 한편으로 디지털 전환 자체가 젠더에 따라 다르게 경험되고 있고, 디지털 세계에서도 기존의 남성중심적인 구조가 재생산되는 경향이 분명한 것이 현실이다. 그러나 다른 한편으로 빅데이터는 2000년대 이후 정체상태에 머물러 있는 양성평등에 한걸음 더 가까이 갈 수 있는 가능성을 내포하고 있기도 하다.


### 인덱싱 

In [17]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex(nodes_sentence) 


In [18]:
print(index)

In [19]:
query_engine = index.as_query_engine(similarity_top_k=5)

In [20]:
response = query_engine.query("디지털 인문학은 인문학을 어떤 방식과 관점으로 다루는 학문인지 알려줘")
print(response)


디지털 인문학은 전통적인 인문학의 방법과 관점을 디지털 기술과 데이터에 적용하여 연구하는 학문으로, 디지털화된 자료를 분석하고 해석함으로써 인문학적인 문제를 탐구하며, 새로운 통찰을 얻고 지식을 확장하는 방식으로 다루는 학문이다.


In [21]:
print(response.source_nodes[0].get_content())

디지털-빅데이터의
시대
인문사회학의
새로운
역할에
대한
고찰
22 
인쇄술이 활용된 14세기 이래 가장 근본적인 변화를 목도하고 있다고 해도 과언이 아닐 것이다. 가장 단순하게는 일반 문서를 컴퓨터가 읽을 수 있는 형태로 스캔하는 것을 포함하는 디지털화가 “디지털 턴”이라는 용어로 구체화되기에 이르렀지만, 특정한 논점에 대해 입증하거나 반박하는 것을 중시하는 전통적인 인문학은 이러한 변화에 대해 소극적이거나 부정적으로 대응해 왔다. 엄청난 규모로 이루어지고 있는 자료의 디지털화에 대해, 그리고 디지털 데이터에 대한 접근이 용이해진 점에 대해서는 경외감을 보이면서도, 이러한 연구들이 분석적인 측면에서의 기여가 분명하지 않다며 “디지털”, “디지털 인문학” 등에 대해 회의적인 시선을 보내고 있는 것이 인문학자들 대부분이 보이는 태도인 듯하다.그러나 현재까지의 각국의 디지털 인문학 관련 논의의 범위와 깊이는 이 분야에 대해 학계 전체의 체계적인 공동대응이 필요함을 보여주고 있다. 먼저 1장에서는 디지털 인문학과 관련되는 여러 쟁점들을 짚어보게 될 것이다. 이 경우 디지털 인문학 연구가 가장 발전한 미국의 최신 연구 및 관련 논의들을 종합적으로 정리한 새런 블럭(Sharon Block)의 논문을 주로 참고하였다. 이 글은 개인 저자인 블럭의 글이기보다 초기 미국사 연구의 최고 권위지인 “William and Mary Quarterly”가 2018년 10월 개최한 워크숍의 논의 결과들을 정리한 글이기 때문에, 이에 근거하여 미국의 인문학자들이 최근의 디지털인문학 연구경향에 대해 어느 정도의 깊이와 범위로 논의를 전개하고 있는지를 파악할 수 있었다.45) 2장에서는 다시 디지털 인문학 연구의 측면에서 앞서가고 있는 미국과 영국사 분야의 연구 및 고고학 분야의 연구를 살펴봄으로써 그동안의 성과에 대해 가늠해보고자 했다. 3장에서는 영국과 독일을 중심으로 하여 각국의 정부가 어떤 방식으로, 그리고 어느 정도로 디지털인문학 연구를 지원하고 있는지 살펴보고, 4장에서는 이에 근

## 텍스트 파일 다루기 

In [1]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["data/paul_graham_essay.txt"])
documents = reader.load_data()

In [2]:
print(len(documents))

1


In [3]:
print(documents)

[Document(id_='de314f02-5bf9-4f51-b641-fe58876cb7a1', embedding=None, metadata={'file_path': 'data\\paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75389, 'creation_date': '2025-03-08', 'last_modified_date': '2024-09-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='What I Worked On\r\n\r\nFebruary 2021\r\n\r\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just cha

In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

llm = OpenAI(model="gpt-4o", temperature=0.2)
embed_model = OpenAIEmbedding()

Settings.llm = llm
Settings.embed_model = embed_model


In [5]:
from llama_index.core.node_parser import SemanticSplitterNodeParser

semantic_splitter = SemanticSplitterNodeParser(
    buffer_size=10, 
    breakpoint_percentile_threshold=80, 
    embed_model=embed_model
)
txt_nodes_semantic=semantic_splitter.get_nodes_from_documents(documents)


In [6]:
print("노드 개수:", len(txt_nodes_semantic))
print(txt_nodes_semantic[10].get_content())


노드 개수: 152
In fact, I decided to write a book about Lisp hacking. It's scary to think how little I knew about Lisp hacking when I started writing that book. But there's nothing like writing a book about something to help you learn it. The book, On Lisp, wasn't published till 1993, but I wrote much of it in grad school.

Computer Science is an uneasy alliance between two halves, theory and systems. The theory people prove things, and the systems people build things. I wanted to build things. 


In [7]:
from llama_index.core import VectorStoreIndex

txt_index = VectorStoreIndex(txt_nodes_semantic)  


In [8]:
#쿼리 엔진 설정하기
txt_query_engine = txt_index.as_query_engine(similarity_top_k=5)

#쿼리 실행하기
txt_response = txt_query_engine.query("저자는 유년 시절에 무엇에 열정을 쏟았어? 자세히 설명해 줘")
print(txt_response)


프로그래밍에 대한 열정을 가지고 있었으며, 간단한 게임을 만들고 모델 로켓이 얼마나 높이 날아갈지 예측하는 프로그램을 작성했습니다. 또한 아버지가 책을 쓸 때 사용한 워드 프로세서도 만들었는데, 이 프로그램은 한 번에 약 2 페이지의 텍스트만 저장할 수 있었기 때문에 아버지는 2 페이지씩 작성한 후 인쇄했습니다. 프로그래밍을 좋아했지만 대학에서는 철학을 공부할 계획이었으나, 대학에 와서 철학 수업이 지루하다고 느껴 다른 분야에서는 무시해도 되는 가장자리 사례들만 남아 있다고 느꼈습니다.


### 크로마 활용하기 

In [8]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

documents = SimpleDirectoryReader(input_files=["data/paul_graham_essay.txt"]).load_data()


In [9]:
db = chromadb.PersistentClient(path="./chroma_db_test")

In [10]:
chroma_collection = db.get_or_create_collection("quickstart_v.1")

In [11]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [12]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)


In [13]:
query_engine = index.as_query_engine()
response = query_engine.query("저자는 유년 시절에 어떤 작업에 열중했어? 자세히 설명해 줘")
print(response)


저자는 유년 시절에 글쓰기와 프로그래밍에 주력했습니다. 학교 외에서 주로 글쓰기와 프로그래밍을 하며, 9학년 때 IBM 1401 컴퓨터를 사용하여 프로그램을 작성하기도 했습니다. 이후에는 마이크로컴퓨터가 등장하면서 컴퓨터 앞에서 직접 프로그램을 입력할 수 있게 되었고, TRS-80와 Apple II와 같은 컴퓨터를 구입하여 프로그래밍을 시작했습니다. 이렇게 프로그래밍을 통해 유년 시절을 보냈으며, 대학에서는 철학을 전공할 계획이었지만 나중에 인공지능으로 전향하게 되었습니다.


##### chroma에 저장된 임베딩 불러오기

In [1]:
# 임베딩 불러오기 

import chromadb

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# 크로마 클라이언트 초기화
db = chromadb.PersistentClient(path="./chroma_db_test")

# 컬렉션 호출
chroma_collection = db.get_or_create_collection("quickstart_v.1")


In [2]:
# 벡터 스토어 설정
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 저장했던 인덱스 로드
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

In [3]:
query_engine = index.as_query_engine()
response = query_engine.query("저자에게 프로그래밍은 어떤 의미인가요")
print(response)


프로그래밍은 저자에게 초기에는 IBM 1401에서의 실험과 TRS-80을 통한 게임 및 워드 프로세서 개발을 통해 컴퓨터와 상호작용하고 창의적인 작업을 할 수 있는 수단으로 보였습니다. 이후에는 인공지능 분야로 발을 들여놓게 되었고, Lisp를 통해 프로그래밍을 통해 새로운 아이디어를 탐구하고 새로운 한계를 경험하는 것으로 여겼습니다.


## CSV 파일 다루기 

In [1]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["data/범죄 발생 장소별 통계_2023.csv"])
documents = reader.load_data()

print("documents 개수:", len(documents))
print(documents[0])


documents 개수: 1
Doc ID: 6b46bd06-e6e3-4169-999f-6e6131e40e88
Text: 강력범죄,살인미수,48,66,38,16,6,0,1,4,8,0,0,0,0,0,2,0,2,17,0,0,0,0,2,1,4
,3,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,1,7,2,2,0,0,0,0,1
,29,4 강력범죄,살인미수등,61,67,45,19,13,1,1,16,40,0,1,3,1,2,0,0,12,13,3,1,1,0,
14,1,10,6,1,1,0,0,0,0,1,5,0,0,3,0,0,0,1,0,0,1,0,3,0,0,1,1,14,3,15,1,3,
0,3,0,3,1,93,7
강력범죄,강도,36,40,29,42,9,1,1,18,42,3,4,13,35,3,0,3,23,45,1,1,...


In [2]:
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import CSVReader

parser = CSVReader()
file_extractor = {".csv": parser}  


In [3]:
documents = SimpleDirectoryReader(
    input_files=["data/범죄 발생 장소별 통계_2023.csv"], 
    file_extractor=file_extractor
).load_data()


In [4]:
print("documents 개수:", len(documents))
print(documents[0])


documents 개수: 1
Doc ID: 0dc0e4cf-0975-4c57-9886-9fe6772eee78
Text: ﻿"범죄대분류, 범죄중분류, 단독주택_다가구_다중, 아파트, 다세대_연립, 오피스텔_원룸, 기타거주시설_기숙사 등,
고속도로, 자동차 전용도로, 일반도로, 통행로_보도_골목길, 백화점, 대형할인점, 슈퍼마켓_소매점, 편의점, 시장_노점,
창고_매장창고한정, 무인상점, 기타상점, 숙박업소_호텔_모텔_여관, 목욕탕_찜질방_사우나, 이발소_미용실, 마사지업소,
공중위생업소_기타, 음식점, 카페, 주점, 단란_유흥주점_나이트_클럽_카바레, 버스터미널_정류소, 지하철역_전철역, 기차역,
여객선터미널, 공항, 버스, 택시, 자가용자동차, 지하철_전철, 기차, 선박, 비행기, 교통수단내_기타, 공연장_극장,
체육시설, 공원_놀...


In [20]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()
semantic_splitter = SemanticSplitterNodeParser(
    buffer_size=10, 
    breakpoint_percentile_threshold=80, 
    embed_model=embed_model
)
csv_nodes_semantic=semantic_splitter.get_nodes_from_documents(documents)


In [21]:
print("노드 개수:", len(csv_nodes_semantic))
print(csv_nodes_semantic[0].get_content())


노드 개수: 1
﻿"범죄대분류, 범죄중분류, 단독주택_다가구_다중, 아파트, 다세대_연립, 오피스텔_원룸, 기타거주시설_기숙사 등, 고속도로, 자동차 전용도로, 일반도로, 통행로_보도_골목길, 백화점, 대형할인점, 슈퍼마켓_소매점, 편의점, 시장_노점, 창고_매장창고한정, 무인상점, 기타상점, 숙박업소_호텔_모텔_여관, 목욕탕_찜질방_사우나, 이발소_미용실, 마사지업소, 공중위생업소_기타, 음식점, 카페, 주점, 단란_유흥주점_나이트_클럽_카바레, 버스터미널_정류소, 지하철역_전철역, 기차역, 여객선터미널, 공항, 버스, 택시, 자가용자동차, 지하철_전철, 기차, 선박, 비행기, 교통수단내_기타, 공연장_극장, 체육시설, 공원_놀이시설, 게임장_(PC)방, 기타_(DVD)방_유원지, 어린이집_유치원, 학교, 도서관, 학원, 기타 교육시설, 금융보험기관, 의료기관, 종교시설, 야외_산야, 해안, 폐가_공터, 공중화장실, 관공서, 군사기지_군사시설, 구금장소, 사회복지시설, 기타, 미상"
강력범죄,살인미수,48,66,38,16,6,0,1,4,8,0,0,0,0,0,2,0,2,17,0,0,0,0,2,1,4,3,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,1,7,2,2,0,0,0,0,1,29,4
강력범죄,살인미수등,61,67,45,19,13,1,1,16,40,0,1,3,1,2,0,0,12,13,3,1,1,0,14,1,10,6,1,1,0,0,0,0,1,5,0,0,3,0,0,0,1,0,0,1,0,3,0,0,1,1,14,3,15,1,3,0,3,0,3,1,93,7
강력범죄,강도,36,40,29,42,9,1,1,18,42,3,4,13,35,3,0,3,23,45,1,1,7,1,7,9,10,15,0,3,0,0,0,0,3,10,0,0,0,0,0,1,1,5,7,1,1,2,0,0,0,12,1,1,19,0,0,3,1,0,0,1,100,3
강력범죄,강간,418,680,624,693,69,0,0,20,34,2,3,0,5,2,0,0,28,1592,3

In [22]:
from llama_index.core import VectorStoreIndex

csv_index = VectorStoreIndex(csv_nodes_semantic)  


In [23]:
csv_query_engine = csv_index.as_query_engine(similarity_top_k=5)

csv_response = csv_query_engine.query("절도범죄가 가장 많이 일어난 장소는 어디야? 수치도 알려줘")
print(csv_response)


절도범죄가 가장 많이 일어난 장소는 "도로"이며, 해당 장소에서 절도범죄가 15,401건 발생했습니다.


In [24]:
csv_response = csv_query_engine.query( "편의점에서는 자주 벌어지는 범죄는 어떤 것들이 있어? 이 중에서 가장 빈번하게 일어난 범죄와 범죄 건수도 알려줘" )
print(csv_response)

편의점에서 자주 발생하는 범죄는 절도범죄, 폭행, 사기 등이 있습니다. 이 중에서 가장 빈번하게 발생하는 범죄는 절도범죄이며, 해당 범죄 건수는 15,401건입니다.


## HWP 파일 다루기 

In [8]:
from llama_index.readers.file import HWPReader

reader = HWPReader()
documents = reader.load_data("data/미국의 통화정책 변화가 외국자본 유출에 미치는 영향_20180528.hwp")

print("도큐먼트 개수: ", len(documents)) 
print(documents[0])


도큐먼트 개수:  1
Doc ID: 41066c48-d1da-4d51-b3b9-4bf2bbd8d4c3
Text: 汤捯    捤獥     氠瑢        漠杳        漠杳        漠杳        漠杳    
2018년 5월 28일(월) 조간   漠杳       최우진 KDI 거시경제연구부 연구위원  (044-550-4053,
wooj.choi@kdi.re.kr)   漠杳       2018년 5월 25일(금) 09:00   漠杳       KDI
홍보팀(044-550-4030, press@kdi.re.kr)   漠杳       桤灧      미국의 통화정책 변화가
외국자본 유출에 미치는 영향 漠杳       최우진 거시경제연구부 연구위원   氠瑢     湯湷      본고는 2018년
상반기...


In [9]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["data/미국의 통화정책 변화가 외국자본 유출에 미치는 영향_20180528.hwp"], encoding='euc-kr' )
documents = reader.load_data()

print("도큐먼트 개수: ", len(documents)) 
print(documents[0].get_content())


도큐먼트 개수:  1
汤捯    捤獥    氠瑢    
漠杳    
漠杳    
漠杳    
漠杳    
2018년 5월 28일(월) 조간
漠杳    
최우진 KDI 거시경제연구부 연구위원
(044-550-4053, wooj.choi@kdi.re.kr)
漠杳    
2018년 5월 25일(금) 09:00
漠杳    
KDI 홍보팀(044-550-4030, press@kdi.re.kr)
漠杳    
桤灧    
미국의 통화정책 변화가 
외국자본 유출에 미치는 영향漠杳    
최우진 거시경제연구부 연구위원
氠瑢    湯湷    
본고는 2018년 상반기 『KDI 경제전망』에 수록될 예정임.

汤捯    捤獥    湯湷    漠杳    연구위원 최우진
汤捯    미국의 통화정책 변화가 외국자본 유출에 미치는 영향
1. 문제제기潴景    潴景    慤桥    慤桥    桤灧    
漠杳    漠杳    
湯慴    
漠杳    漠杳    
湯慴    
漠杳    漠杳    
미국의 통화정책 변화가 외국자본 유출에 미치는 영향
漠杳    漠杳    
미국의 통화정책 변화가 외국자본 유출에 미치는 영향
漠杳     미국의 금리인상이 본격화될 것으로 예상되는 가운데, 신흥국 불안에 따른 국제금융시장의 변동성도 확대되고 있어 급격한 외국자본 유출에 대한 우려가 확대됨.
漠杳     2008년 금융위기 이후 ‘제로금리’를 유지하였던 미국의 통화당국은 2015년 12월을 시작으로 현재까지 정책금리를 25bp씩 여섯 차례 인상한 바 있음.
漠杳     최근에는 아르헨티나가 자국 경제불안에 따른 대외신뢰도 저하 및 통화가치 하락으로 IMF에 구제금융을 신청하면서 미국의 금리인상에 따른 신흥국의 외국자본 유출 우려가 높아지는 상황
漠杳     외국자본 유출은 형태에 따라 실물경제에 미치는 영향이 다르게 나타날 수 있음.
漠杳     서브프라임

In [10]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

llm = OpenAI(model="gpt-4o", temperature=0.2)
embed_model = OpenAIEmbedding()

Settings.llm = llm
Settings.embed_model = embed_model


In [11]:
from llama_index.core.node_parser import SemanticSplitterNodeParser

semantic_splitter = SemanticSplitterNodeParser(
    buffer_size=10, 
    breakpoint_percentile_threshold=80, 
    embed_model=embed_model
)
hwp_nodes_semantic=semantic_splitter.get_nodes_from_documents(documents)


In [12]:
print("노드의 개수:", len(hwp_nodes_semantic))
print(hwp_nodes_semantic[11].get_content())


노드의 개수: 13
漠杳     한편, 정치/지정학적 위험의 경우에는 채권투자를 중심으로 GDP 대비 0.10%(전체 외국자본 대비 0.13%) 규모의 유출이 발생하는 것으로 추정됨.
╺ 남북 정상회담 등으로 지정학적 위험이 다소 완화됨에 따라 국제금융시장의 변동성 확대 등에 따른 채권투자의 유출을 일정 부분 완충시킬 가능성을 시사
4. 요약과 시사점
漠杳     미국의 금리인상이 본격화될 것으로 예상되는 가운데, 신흥국 불안에 따라 국제금융시장의 변동성도 확대되고 있어 급격한 외국자본의 유출에 대한 우려가 확대됨.
漠杳     외국자본 유출은 그 형태에 따라 실물경제에 반영되는 모습이 다르게 나타날 수 있어, 미국 금리인상이 형태별 외국자본 유출에 미치는 영향을 살펴볼 필요가 있음.
漠杳     미국 금리의 인상은 부채성 자금(채권 및 차입 투자)을 중심으로 외국자본을 유출시킬 가능성이 있으나, 그 규모는 우리 경제의 규모와 외환보유액 등 외환건전성 상황을 감안할 때 미미한 것으로 평가됨.
漠杳     실증분석 결과, 미국의 통화정책 변화에 따른 금리인상은 주로 차입투자와 채권투자 자금을 중심으로 외국자본의 유출을 유발할 수 있으나 이는 GDP 대비 0.38% 정도로 미미한 수준이라고 평가할 수 있음.



In [13]:
from llama_index.core import VectorStoreIndex

hwp_index = VectorStoreIndex(hwp_nodes_semantic)  

hwp_query_engine = hwp_index.as_query_engine(similarity_top_k=5)

hwp_response = hwp_query_engine.query("미국 금리가 올라가면 한국 금융 시장은 어떤 영향을 받아? 한국어로 정리해 줘")
print(hwp_response)


미국 금리가 인상되면 한국 금융 시장에서는 주로 부채성 자금, 즉 채권 및 차입 투자를 중심으로 외국자본의 유출이 발생할 가능성이 있습니다. 그러나 이러한 자본 유출의 규모는 한국 경제의 규모와 외환보유액 등을 고려할 때 상대적으로 미미한 수준으로 평가됩니다. 외국자본 유출은 그 형태에 따라 실물경제에 미치는 영향이 다르게 나타날 수 있으며, 미국 금리 인상은 특히 차입투자와 채권투자 자금을 중심으로 외국자본의 유출을 유발할 수 있습니다. 다만, 전체 외국자본 유출은 GDP 대비 0.38% 정도로 미미한 수준으로 평가됩니다.


In [50]:
print("소스노드의 수: ", len(hwp_response.source_nodes))
print(hwp_response.source_nodes[0].get_content())


소스노드의 수:  5
Louis; Bloomberg.
漠杳     한편, 한국과 미국 금융시장의 구조적 차이를 반영하기 위해 모형을 재설정하여 분석을 시도한 결과에서도, 미국의 금리인상이 외국자본의 유출에 미치는 영향은 미미한 것으로 나타남.
漠杳     원화와 미 달러화 표시 자산들의 수익률이 단순 비교될 수 없다는 점에서, 한미 금리차가 아닌 각국 금리를 독립적인 설명변수로 사용하였음.

